In [207]:
import sys
sys.path.append("../src")

import pandas as pd
from etf_transformations import *
from pathlib import Path

from catboost import CatBoostClassifier
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from scipy.stats import randint, uniform
from sklearn.metrics import roc_auc_score, accuracy_score



In [208]:
xlf = pd.read_csv("../data/preprocessed/XLF_preprocessed.csv")
xlf.head()

,Date,Price,Return,Sign
0,2018-03-20,24.879965,NaN,NaN
1,2018-03-21,24.871304,-0.000348,-1.0
2,2018-03-22,23.953358,-0.036908,-1.0
3,2018-03-23,23.225918,-0.030369,-1.0
4,2018-03-26,23.979336,0.032439,1.0


In [209]:
news_df = pd.read_csv("../data/preprocessed/final_data_with_embeddings_without_aggregations.csv")
news_df.head()

,Date,Headlines,Newspaper,XLE,XLF,XLK,XLV,XLY,positive,neutral,...,emb_374,emb_375,emb_376,emb_377,emb_378,emb_379,emb_380,emb_381,emb_382,emb_383
0,2018-03-20,"Cramer: Trump, the Fed and the cloud can get t...",cnbc,0,0,1,0,0,0.381990,0.606314,...,0.085011,-0.048258,-0.054372,-0.046042,-0.058606,-0.016788,0.004511,-0.123710,0.027060,0.097270
1,2018-03-20,G20 financial leaders say need more dialogue o...,reuters,0,0,0,0,0,0.259833,0.077557,...,0.066003,0.030541,-0.024649,-0.044539,-0.018354,-0.018619,0.003425,-0.027990,-0.001139,-0.002184
2,2018-03-20,Index provider MSCI says it is reviewing Faceb...,reuters,0,0,0,0,0,0.018125,0.884106,...,-0.019214,0.047262,-0.019467,0.056820,0.032496,0.017863,0.107006,-0.092001,-0.023388,-0.012118
3,2018-03-20,UK's Cambridge University questions Facebook a...,reuters,0,0,0,0,0,0.032822,0.651042,...,-0.008723,-0.009251,0.048255,0.079781,-0.097413,-0.071449,0.053479,-0.088406,0.024191,-0.039716
4,2018-03-20,"Exclusive: On Amazon, a quarter of merchants' ...",reuters,0,0,0,0,1,0.038724,0.937915,...,0.058902,-0.006282,-0.009820,-0.062463,0.025669,0.082309,0.057419,-0.089949,0.036099,0.055160


In [210]:
# PREPROCESSED_DIR = Path("..") / "data" / "preprocessed"
# csv_files = list(PREPROCESSED_DIR.glob("*_preprocessed.csv"))

# csv_files


In [211]:
# def merge_sectors_with_news(csv_path):
#     ticker = csv_path.stem.split("_")[0]
#     out_dir = Path("../data/preprocessed/")
    
#     df = pd.read_csv(csv_path)
#     df = df.copy()
#     df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
#     df = df.sort_values("Date")

#     news_df = pd.read_csv("../data/preprocessed/final_data_with_embeddings_without_aggregations.csv")
#     news_df = news_df.copy()
#     news_df["Date"] = pd.to_datetime(news_df["Date"]).dt.normalize()
#     news_df = news_df.sort_values("Date")
#     #news_df = news_df[news_df[ticker] == 1].copy()


#     date_flags = (news_df[["Date", "is_trading_day"]].drop_duplicates("Date").sort_values("Date").reset_index(drop=True))
#     date_flags["TradingDate"] = date_flags["Date"].where(date_flags["is_trading_day"]==1)
#     date_flags["TradingDate"] = date_flags["TradingDate"].bfill()

#     news_df = news_df.merge(date_flags[["Date", "TradingDate"]], on="Date", how="left").dropna(subset=["TradingDate"])
#     news_df["days_to_trading"] = (news_df["TradingDate"] - news_df["Date"]).dt.days
#     news_df["is_same_day_news"] = (news_df["days_to_trading"] == 0).astype(int)
#     news_df["is_weekend_news"] = (news_df["Date"].dt.weekday >= 5).astype(int)

#     news_df = news_df.rename(columns={"Date": "PublicationDate"})
#     df = df.rename(columns={"Date": "TradingDate"})

#     dfs_combined = news_df.merge(df, on="TradingDate", how="left")

#     subdir = Path(out_dir) / "headlines_with_sentiment" 
#     subdir.mkdir(parents=True, exist_ok=True)
#     out_path = subdir / f"{ticker}_headlines_separately.csv" 

#     dfs_combined.to_csv(out_path, index=False)
#     print(f'{ticker} done.')
#     return out_path


# for i in csv_files:
#     merge_sectors_with_news(i)

In [212]:
df = pd.read_csv("../data/preprocessed/headlines_with_sentiment/XLF_headlines_separately.csv")

df.head()


,PublicationDate,Headlines,Newspaper,XLE,XLF,XLK,XLV,XLY,positive,neutral,...,emb_381,emb_382,emb_383,TradingDate,days_to_trading,is_same_day_news,is_weekend_news,Price,Return,Sign
0,2018-03-20,"Cramer: Trump, the Fed and the cloud can get t...",cnbc,0,0,1,0,0,0.381990,0.606314,...,-0.123710,0.027060,0.097270,2018-03-20,0,1,0,24.879965,NaN,NaN
1,2018-03-20,Exxon eyes Gulf of Mexico plastics plant to me...,reuters,1,0,0,0,0,0.867670,0.123859,...,-0.035563,0.088698,0.106409,2018-03-20,0,1,0,24.879965,NaN,NaN
2,2018-03-20,Weinstein Co says its bankruptcy filing won't ...,reuters,0,0,0,0,0,0.012923,0.036247,...,-0.123302,0.105105,0.023598,2018-03-20,0,1,0,24.879965,NaN,NaN
3,2018-03-20,Japan gets its way at G20 on warning against r...,reuters,0,0,0,0,0,0.506586,0.176257,...,-0.121720,-0.047011,0.059398,2018-03-20,0,1,0,24.879965,NaN,NaN
4,2018-03-20,Social media stocks tumble as Wall Street fear...,reuters,0,1,0,0,0,0.009973,0.050827,...,-0.061051,-0.069362,0.025928,2018-03-20,0,1,0,24.879965,NaN,NaN


In [213]:
df["TradingDate"] = pd.to_datetime(df["TradingDate"]).dt.normalize()
df = df.sort_values("TradingDate")

daily_sign = (
    df[["TradingDate", "Sign"]]
    .drop_duplicates("TradingDate")
    .sort_values("TradingDate")
)

daily_sign["Sign_next_day"] = daily_sign["Sign"].shift(-1)

df = df.merge(
    daily_sign[["TradingDate", "Sign_next_day"]],
    on="TradingDate",
    how="left"
)

df.head()

,PublicationDate,Headlines,Newspaper,XLE,XLF,XLK,XLV,XLY,positive,neutral,...,emb_382,emb_383,TradingDate,days_to_trading,is_same_day_news,is_weekend_news,Price,Return,Sign,Sign_next_day
0,2018-03-20,"Cramer: Trump, the Fed and the cloud can get t...",cnbc,0,0,1,0,0,0.381990,0.606314,...,0.027060,0.097270,2018-03-20,0,1,0,24.879965,NaN,NaN,-1.0
1,2018-03-20,"G20 talks on trade 'constructive,' no concern ...",reuters,0,0,0,0,0,0.144710,0.831287,...,-0.076166,0.020394,2018-03-20,0,1,0,24.879965,NaN,NaN,-1.0
2,2018-03-20,G20 financial leaders say need more dialogue o...,reuters,0,0,0,0,0,0.259833,0.077557,...,-0.001139,-0.002184,2018-03-20,0,1,0,24.879965,NaN,NaN,-1.0
3,2018-03-20,Index provider MSCI says it is reviewing Faceb...,reuters,0,0,0,0,0,0.018125,0.884106,...,-0.023388,-0.012118,2018-03-20,0,1,0,24.879965,NaN,NaN,-1.0
4,2018-03-20,UK's Cambridge University questions Facebook a...,reuters,0,0,0,0,0,0.032822,0.651042,...,0.024191,-0.039716,2018-03-20,0,1,0,24.879965,NaN,NaN,-1.0


In [214]:
columns_to_drop = [
    "Headlines",
    "Price",
    "Return",
    "Sign",
    "XLE",
    "XLK",
    "XLV",
    "XLY",
    "avg_positive_global",
    "avg_neutral_global",
    "avg_negative_global",
    "n_headlines",
    "sent_index_global",
    "row_id"
]

In [215]:
df = df.drop(columns=columns_to_drop)

for i in df.columns:
    print(i)


PublicationDate
Newspaper
XLF
positive
neutral
negative
finbert_label
finbert_confidence
is_trading_day
emb_0
emb_1
emb_2
emb_3
emb_4
emb_5
emb_6
emb_7
emb_8
emb_9
emb_10
emb_11
emb_12
emb_13
emb_14
emb_15
emb_16
emb_17
emb_18
emb_19
emb_20
emb_21
emb_22
emb_23
emb_24
emb_25
emb_26
emb_27
emb_28
emb_29
emb_30
emb_31
emb_32
emb_33
emb_34
emb_35
emb_36
emb_37
emb_38
emb_39
emb_40
emb_41
emb_42
emb_43
emb_44
emb_45
emb_46
emb_47
emb_48
emb_49
emb_50
emb_51
emb_52
emb_53
emb_54
emb_55
emb_56
emb_57
emb_58
emb_59
emb_60
emb_61
emb_62
emb_63
emb_64
emb_65
emb_66
emb_67
emb_68
emb_69
emb_70
emb_71
emb_72
emb_73
emb_74
emb_75
emb_76
emb_77
emb_78
emb_79
emb_80
emb_81
emb_82
emb_83
emb_84
emb_85
emb_86
emb_87
emb_88
emb_89
emb_90
emb_91
emb_92
emb_93
emb_94
emb_95
emb_96
emb_97
emb_98
emb_99
emb_100
emb_101
emb_102
emb_103
emb_104
emb_105
emb_106
emb_107
emb_108
emb_109
emb_110
emb_111
emb_112
emb_113
emb_114
emb_115
emb_116
emb_117
emb_118
emb_119
emb_120
emb_121
emb_122
emb_123
emb_124
emb_12

In [216]:
df["PublicationDate"] = pd.to_datetime(df["PublicationDate"])
df["TradingDate"] = pd.to_datetime(df["TradingDate"])

print(df["PublicationDate"].min(), df["PublicationDate"].max())
print(df["TradingDate"].min(), df["TradingDate"].max())


2018-03-20 00:00:00 2020-07-17 00:00:00
2018-03-20 00:00:00 2020-07-17 00:00:00


In [217]:
df = df.dropna(subset=["Sign_next_day"])
df["Sign_next_day"] = (df["Sign_next_day"] == 1).astype(int)
df.head()

,PublicationDate,Newspaper,XLF,positive,neutral,negative,finbert_label,finbert_confidence,is_trading_day,emb_0,...,emb_379,emb_380,emb_381,emb_382,emb_383,TradingDate,days_to_trading,is_same_day_news,is_weekend_news,Sign_next_day
0,2018-03-20,cnbc,0,0.381990,0.606314,0.011696,neutral,0.606314,1,-0.027789,...,-0.016788,0.004511,-0.123710,0.027060,0.097270,2018-03-20,0,1,0,0
1,2018-03-20,reuters,0,0.144710,0.831287,0.024004,neutral,0.831287,1,-0.057689,...,0.036421,-0.017355,-0.058831,-0.076166,0.020394,2018-03-20,0,1,0,0
2,2018-03-20,reuters,0,0.259833,0.077557,0.662610,negative,0.662610,1,-0.047038,...,-0.018619,0.003425,-0.027990,-0.001139,-0.002184,2018-03-20,0,1,0,0
3,2018-03-20,reuters,0,0.018125,0.884106,0.097769,neutral,0.884106,1,-0.027929,...,0.017863,0.107006,-0.092001,-0.023388,-0.012118,2018-03-20,0,1,0,0
4,2018-03-20,reuters,0,0.032822,0.651042,0.316137,neutral,0.651042,1,-0.010858,...,-0.071449,0.053479,-0.088406,0.024191,-0.039716,2018-03-20,0,1,0,0


In [218]:
df.tail()

,PublicationDate,Newspaper,XLF,positive,neutral,negative,finbert_label,finbert_confidence,is_trading_day,emb_0,...,emb_379,emb_380,emb_381,emb_382,emb_383,TradingDate,days_to_trading,is_same_day_news,is_weekend_news,Sign_next_day
51045,2020-07-16,reuters,0,0.133518,0.853361,0.013120,neutral,0.853361,1,0.007655,...,-0.010502,0.064772,-0.055863,0.077422,0.026832,2020-07-16,0,1,0,0
51046,2020-07-16,reuters,1,0.027702,0.082280,0.890018,negative,0.890018,1,0.051069,...,-0.073149,0.050732,-0.113782,0.086610,-0.012189,2020-07-16,0,1,0,0
51047,2020-07-16,reuters,0,0.029390,0.801751,0.168859,neutral,0.801751,1,0.003858,...,0.023498,0.002181,-0.034257,0.019681,0.039421,2020-07-16,0,1,0,0
51048,2020-07-16,reuters,0,0.020707,0.092245,0.887048,negative,0.887048,1,0.001260,...,0.012409,0.004730,-0.035170,-0.027257,0.031442,2020-07-16,0,1,0,0
51049,2020-07-16,reuters,1,0.833721,0.049213,0.117067,positive,0.833721,1,0.006331,...,-0.043842,0.034886,-0.151829,-0.031995,0.032237,2020-07-16,0,1,0,0


In [219]:
split_date = pd.Timestamp("2020-01-01")

train_df = df[df["TradingDate"] < split_date].copy()
test_df  = df[df["TradingDate"] >= split_date].copy()



In [220]:
columns_to_exclude = [
    "Sign_next_day",
    "PublicationDate",
    "TradingDate",
]

X_train = train_df.drop(columns=columns_to_exclude)
X_test  = test_df.drop(columns=columns_to_exclude)

y_train = train_df["Sign_next_day"]
y_test  = test_df["Sign_next_day"]



In [221]:
def class_balance(y, name=""):
    counts = y.value_counts().sort_index()
    frac = y.value_counts(normalize=True).sort_index()
    out = pd.DataFrame({"count": counts, "share": frac})
    print(f"\n{name} class balance:")
    print(out)

class_balance(df["Sign_next_day"], "ALL")
class_balance(y_train, "TRAIN")
class_balance(y_test, "TEST")



ALL class balance:
               count     share
Sign_next_day                 
0              24387  0.477708
1              26663  0.522292

TRAIN class balance:
               count     share
Sign_next_day                 
0              17371  0.465973
1              19908  0.534027

TEST class balance:
               count     share
Sign_next_day                 
0               7016  0.509476
1               6755  0.490524


In [222]:
df["Sign_next_day"].value_counts()
df["Sign_next_day"].value_counts(normalize=True)


Sign_next_day
1    0.522292
0    0.477708
Name: proportion, dtype: float64

In [223]:
print("n_train:", len(train_df), "n_test:", len(test_df))
print("n_features:", X_train.shape[1])


n_train: 37279 n_test: 13771
n_features: 395


In [224]:
print("n_train:", len(train_df), "n_test:", len(test_df))
print("n_features:", X_train.shape[1])


n_train: 37279 n_test: 13771
n_features: 395


In [225]:
X_train.isna().mean().sort_values(ascending=False).head(10)


is_weekend_news       0.0
Newspaper             0.0
XLF                   0.0
positive              0.0
neutral               0.0
negative              0.0
finbert_label         0.0
finbert_confidence    0.0
is_trading_day        0.0
emb_0                 0.0
dtype: float64

In [226]:
cat_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()
print("Categorical columns:", cat_cols)

Categorical columns: ['Newspaper', 'finbert_label']


In [227]:
for c in cat_cols:
    X_train[c] = X_train[c].astype(str)
    X_test[c]  = X_test[c].astype(str)

cat_features = [X_train.columns.get_loc(c) for c in cat_cols]

In [230]:
base_model = CatBoostClassifier(
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=0,
    allow_writing_files=False,
    cat_features=cat_features
)

param_distributions = {
    "depth": randint(3, 7),
    "learning_rate": uniform(0.01, 0.14),
    "iterations": randint(300, 1200),
    "l2_leaf_reg": uniform(1.0, 14.0),
    "random_strength": uniform(0.0, 5.0),
    "bagging_temperature": uniform(0.0, 1.0),
    "border_count": randint(32, 255),
    "bootstrap_type": ["Bayesian"],
    "grow_policy": ["SymmetricTree", "Depthwise"],
}


tscv = TimeSeriesSplit(n_splits=5)

search = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_distributions,
    n_iter=40,
    scoring="roc_auc",
    cv=tscv,
    n_jobs=-1,
    random_state=42,
    verbose=2
)

search.fit(X_train, y_train)

print("Best AUC (CV):", search.best_score_)
print("Best params:", search.best_params_)


Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best AUC (CV): 0.5091128143296125
Best params: {'bagging_temperature': np.float64(0.7473201101373809), 'bootstrap_type': 'Bayesian', 'border_count': 142, 'depth': 3, 'grow_policy': 'Depthwise', 'iterations': 516, 'l2_leaf_reg': np.float64(4.933083135623331), 'learning_rate': np.float64(0.08597745164215478), 'random_strength': np.float64(0.7046211248738132)}


In [231]:
best_model = search.best_estimator_

proba_test = best_model.predict_proba(X_test)[:, 1]
pred_test  = (proba_test >= 0.5).astype(int)

print("Test AUC:", roc_auc_score(y_test, proba_test))
print("Test ACC:", accuracy_score(y_test, pred_test))

Test AUC: 0.4941722188133795
Test ACC: 0.4891438530244717


In [232]:
best_params = search.best_params_.copy()

final_model = CatBoostClassifier(
    **best_params,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=0,
    allow_writing_files=False,
    cat_features=cat_features
)

final_model.fit(X_train, y_train)

proba_test = final_model.predict_proba(X_test)[:, 1]
pred_test  = (proba_test >= 0.5).astype(int)


print("Final test AUC:", roc_auc_score(y_test, proba_test))
print("Final test ACC:", accuracy_score(y_test, pred_test))


Final test AUC: 0.4941722188133795
Final test ACC: 0.4891438530244717
